# Test to ensure modifications to function do not change evaluated value

Beginning modification of the function to handle case where N is not equal to 1.

In [1]:
import math
from numba import cuda
import ZMCIntegral
import time
import numpy as np
import scipy
import scipy.special
from scipy.integrate import quad

Define constants in function

In [2]:
mu = 0.1  # Fermi-level
hOmg = 0.5  # Photon energy eV
a = 4  # AA
A = 4  # hbar^2/(2m)=4 evAA^2 (for free electron mass)
rati = 0.3  # ratio
eE0 = rati * ((hOmg) ** 2) / (2 * np.sqrt(A * mu))
# print(eE0)
Gamm = 0.005  # Gamma in eV.
KT = 1 * 10 ** (-6)
shift = A * (eE0 / hOmg) ** 2

The original function is declared below.

In [3]:
def Ds(kx, ky, qx, qy):
    N = 3
    dds = 0
    ds = 0
    ek = A * (math.sqrt((kx) ** 2 + (ky) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((kx + qx) ** 2 + (ky + qy) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((kx) ** 2 + (ky) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((kx + qx) ** 2 + (ky + qy) ** 2) / hOmg ** 2
    
    # print('xk = ', xk)
    # print('xkq = ', xkq)

    sing = np.arange(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
    taninv1kp = 2 * np.arctan2(Gamm, ek - hOmg / 2 + hOmg * sing)
    taninv1kqp = 2 * np.arctan2(Gamm, ekq - hOmg / 2 + hOmg * sing)
    taninv1km = 2 * np.arctan2(Gamm, ek + hOmg / 2 + hOmg * sing)
    taninv1kqm = 2 * np.arctan2(Gamm, ekq + hOmg / 2 + hOmg * sing)

    lg1kp = complex(0, 1) * np.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * sing) ** 2)
    lg1kqp = complex(0, 1) * np.log(Gamm ** 2 + (ekq - hOmg / 2 + hOmg * sing) ** 2)
    lg1km = complex(0, 1) * np.log(Gamm ** 2 + (ek + hOmg / 2 + hOmg * sing) ** 2)
    lg1kqm = complex(0, 1) * np.log(Gamm ** 2 + (ekq + hOmg / 2 + hOmg * sing) ** 2)

    ferp = np.heaviside(mu - hOmg / 2 - hOmg * sing, 0)
    ferm = np.heaviside(mu + hOmg / 2 - hOmg * sing, 0)

    dbl = np.arange(-(N - 1), (N - 1) + 1, 1)
    taninv2k = 2 * np.arctan2(Gamm, ek - mu + hOmg * dbl)
    taninv2kq = 2 * np.arctan2(Gamm, ekq - mu + hOmg * dbl)

    lg2k = complex(0, 1) * np.log(Gamm ** 2 + (ek - mu + hOmg * dbl) ** 2)
    lg2kq = complex(0, 1) * np.log(Gamm ** 2 + (ekq - mu + hOmg * dbl) ** 2)

    besk = scipy.special.jv(dbl, xk)
    beskq = scipy.special.jv(dbl, xkq)

    fac1 = ek - ekq + hOmg * dbl
    fac2 = fac1 + 2 * complex(0, 1) * Gamm
    fac3 = fac2 - ek + ekq
    
    # DEBUGGING
    # print('taninv2k = ', taninv2k)
    # print('taninv1kp = ', taninv1kp)
    # print('lg2k = ', lg2k)
    # print('lg1kp = ', lg1kp)
    # print('fac3 = ', fac3)
    # print('besk = ', besk)
    # print('beskq = ', beskq)
    
    for n in range(0, N):
        for alpha in range(0, N):
            for beta in range(0, N):
                for gamma in range(0, N):
                    for s in range(0, N):
                        for l in range(0, N):
                            p1p = fac1[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])
                            p2p = fac2[alpha - gamma + N - 1] * (
                                    taninv1kp[beta] - taninv2k[s + beta] + lg1kp[beta] - lg2k[s + beta])
                            p3p = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            p1m = fac1[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            p2m = fac2[alpha - gamma + N - 1] * (
                                    taninv1km[beta] - taninv2k[s + beta] + lg1km[beta] - lg2k[s + beta])

                            p3m = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            d1 = -2 * complex(0, 1) * fac1[beta - gamma + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     alpha - beta + N - 1]
                            
                            omint1p = ferp[s] * ((p1p + p2p + p3p) / d1)

                            omint1m = ferm[s] * ((p1m + p2m + p3m) / d1)

                            bess1 = beskq[gamma - n + N - 1] * beskq[gamma - l + N - 1] * besk[beta - l + N - 1] * besk[
                                beta - s + N - 1] * besk[alpha - s + N - 1] * besk[alpha - n + N - 1]

                            grgl = bess1 * (omint1p - omint1m)

                            pp1p = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            pp2p = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqp[beta] + taninv2kq[s + beta] + lg1kqp[beta] - lg2kq[
                                s + beta])

                            pp3p = fac3[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])

                            pp1m = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            pp2m = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqm[beta] + taninv2kq[s + beta] + lg1kqm[beta] - lg2kq[
                                s + beta])

                            pp3m = fac3[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            d2 = -2 * complex(0, 1) * fac1[alpha - beta + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     beta - gamma + N - 1]

                            omint2p = ferp[s] * ((pp1p + pp2p + pp3p) / d2)

                            omint2m = ferm[s] * ((pp1m + pp2m + pp3m) / d2)

                            bess2 = beskq[gamma - n + N - 1] * beskq[gamma - s + N - 1] * beskq[beta - s + N - 1] * \
                                    beskq[beta - l + N - 1] * besk[alpha - l + N - 1] * besk[alpha - n + N - 1]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + 2 * Gamm * (grgl + glga)
                            #DEBUG
                            # print('dds = ', dds)
                            # print('bess1=',  bess1)
    return dds

In [4]:
def my_Besselv(v, z):
    # WILL NOT WORK IF v IS NOT AN INTEGER
    # Conditional to handle case of negative v.
    if(v < 0):
        v = abs(v)
        resultsign = (-1) ** v
    else:
        resultsign = 1
    result = 0    
    # Loop to construct Bessel series sum.
    for n in range(0,20):
        sign = (-1)**n
        exp = 2 * n + v
        term = z ** exp
        r = n + v + 1
        if(r == 0):
            r = 1e-15
        denom = math.gamma(r)
        denom = denom * math.factorial(n)
        denom = denom * (2 ** exp)
        term = term / denom * sign
        # print('for ', n, ': ',term)
        result = result + term
        
    return result * resultsign
        
    
def myHeaviside(z):
    # Wrote this Heaviside expression with it cast in cuda to avoid error below.
    if z <= 0 :
	    return 0
    else :
	    return 1
    
    
def modDsN2(x):
    N = 3
    dds = 0
    ds = 0
    ek = A * (math.sqrt((x[0]) ** 2 + (x[1]) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((x[0] + x[2]) ** 2 + (x[1] + 0) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((x[0]) ** 2 + (x[1]) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((x[0] + x[2]) ** 2 + (x[1] + 0) ** 2) / hOmg ** 2

    # print('modxk = ', xk)
    # print('modxkq = ', xkq)
    
    j = 0
    i = -(N - 1) / 2
    
    taninv1kp = np.empty(N)
    taninv1kqp = np.empty(N)
    taninv1km = np.empty(N)
    taninv1kqm = np.empty(N)

    lg1kp = np.empty(N, dtype='complex')
    lg1kqp = np.empty(N, dtype='complex')
    lg1km = np.empty(N, dtype='complex')
    lg1kqm = np.empty(N, dtype='complex')

    ferp = np.empty(N)
    ferm = np.empty(N)
    
    while(i < ((N - 1) / 2 + 1)):
        taninv1kp[j] = 2 * math.atan2(Gamm, ek - hOmg / 2 + hOmg * i)
        taninv1kqp[j] = 2 * math.atan2(Gamm, ekq - hOmg / 2 + hOmg * i)
        taninv1km[j] = 2 * math.atan2(Gamm, ek + hOmg / 2 + hOmg * i)
        taninv1kqm[j] = 2 * math.atan2(Gamm, ekq + hOmg / 2 + hOmg * i)

        lg1kp[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * i) ** 2)
        lg1kqp[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq - hOmg / 2 + hOmg * i) ** 2)
        lg1km[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek + hOmg / 2 + hOmg * i) ** 2)
        lg1kqm[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq + hOmg / 2 + hOmg * i) ** 2)

        ferp[j] = myHeaviside(mu - hOmg / 2 - hOmg * i)
        ferm[j] = myHeaviside(mu + hOmg / 2 - hOmg * i)
        j = j + 1
        i = i + 1
        
    size_dbl = 2 * N - 1
    
    taninv2k = np.empty(size_dbl)
    taninv2kq = np.empty(size_dbl)

    lg2k = np.empty(size_dbl, dtype='complex')
    lg2kq = np.empty(size_dbl, dtype='complex')

    besk = np.empty(size_dbl)
    beskq = np.empty(size_dbl)

    fac1 = np.empty(size_dbl)
    fac2 = np.empty(size_dbl, dtype='complex')
    fac3 = np.empty(size_dbl, dtype='complex')

    j = 0
    for i in range(-(N - 1), N, 1):
        taninv2k[j] = 2 * math.atan2(Gamm, ek - mu + hOmg * i)
        taninv2kq[j] = 2 * math.atan2(Gamm, ekq - mu + hOmg * i)

        lg2k[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek - mu + hOmg * i) ** 2)
        lg2kq[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq - mu + hOmg * i) ** 2)

        besk[j] = my_Besselv(i, xk)
        beskq[j] = my_Besselv(i, xkq)

        fac1[j] = ek - ekq + hOmg * i
        fac2[j] = fac1[j] + 2 * complex(0, 1) * Gamm
        fac3[j] = fac2[j] - ek + ekq
        j = j + 1
    
    # debug statements
    # print('taninv2k = ', taninv2k)
    # print('taninv1kp = ', taninv1kp)
    # print('lg2k = ', lg2k)
    # print('lg1kp = ', lg1kp)
    # print('fac3 = ', fac3)
    # print('modbesk = ', besk)
    # print('modbeskq = ', beskq)
    
    for n in range(0, N):
        for alpha in range(0, N):
            for beta in range(0, N):
                for gamma in range(0, N):
                    for s in range(0, N):
                        for l in range(0, N):
                            p1p = fac1[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])
                            p2p = fac2[alpha - gamma + N - 1] * (
                                    taninv1kp[beta] - taninv2k[s + beta] + lg1kp[beta] - lg2k[s + beta])
                            p3p = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            p1m = fac1[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            p2m = fac2[alpha - gamma + N - 1] * (
                                    taninv1km[beta] - taninv2k[s + beta] + lg1km[beta] - lg2k[s + beta])

                            p3m = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            d1 = -2 * complex(0, 1) * fac1[beta - gamma + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     alpha - beta + N - 1]

                            omint1p = ferp[s] * ((p1p + p2p + p3p) / d1)

                            omint1m = ferm[s] * ((p1m + p2m + p3m) / d1)

                            bess1 = beskq[gamma - n + N - 1] * beskq[gamma - l + N - 1] * besk[beta - l + N - 1] * besk[
                                beta - s + N - 1] * besk[alpha - s + N - 1] * besk[alpha - n + N - 1]

                            grgl = bess1 * (omint1p - omint1m)

                            pp1p = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            pp2p = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqp[beta] + taninv2kq[s + beta] + lg1kqp[beta] - lg2kq[
                                s + beta])

                            pp3p = fac3[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])

                            pp1m = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            pp2m = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqm[beta] + taninv2kq[s + beta] + lg1kqm[beta] - lg2kq[
                                s + beta])

                            pp3m = fac3[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            d2 = -2 * complex(0, 1) * fac1[alpha - beta + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     beta - gamma + N - 1]

                            omint2p = ferp[s] * ((pp1p + pp2p + pp3p) / d2)

                            omint2m = ferm[s] * ((pp1m + pp2m + pp3m) / d2)

                            bess2 = beskq[gamma - n + N - 1] * beskq[gamma - s + N - 1] * beskq[beta - s + N - 1] * \
                                    beskq[beta - l + N - 1] * besk[alpha - l + N - 1] * besk[alpha - n + N - 1]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + 2 * Gamm * (grgl + glga)
                            
                            #DEBUG
                            # print('moddds = ', dds)
                            # print('modbess1=',  bess1)
    return dds.real

# The modified function

The function is vectorized again to increase efficiency.

The custom Bessel function should now handle any-order*, first-kind Bessel function. *(Integer-order!)

NOTE: machine epsilon value ~E-16 implies that a relative error below E-15 is nonsense. The limit rests around here.

In [5]:
# Make error array.
relerror = np.zeros(900)

print('Comparing modified version and original')
print('================================================================================================')
print(' kx  | ky  | qx  | qy  | Ds          | modDs  ')
print('================================================================================================')
for i in range(0, 75, 5):
    for j in range(0, 75, 5):
        for k in range(1, 4, 1):
            xin = [i/100, j/100, k/100]
            ds_result = Ds(i/100, j/100, k/100, 0).real
            modds_result = modDsN2(xin)
            print('%2.1f  | %2.1f | %2.1f | 0   |'%(i/100, j/100, k/100), ds_result, ' | ', modds_result)
            relerror[i+j+k-1] = abs((modds_result / ds_result.real) - 1)
            

Comparing modified version and original
 kx  | ky  | qx  | qy  | Ds          | modDs  
0.0  | 0.0 | 0.0 | 0   | -4.853670387063359  |  -4.853670387063359
0.0  | 0.0 | 0.0 | 0   | -5.0305425671111665  |  -5.0305425671111665
0.0  | 0.0 | 0.0 | 0   | -5.414388461173376  |  -5.414388461173376
0.0  | 0.1 | 0.0 | 0   | -8.281695505674838  |  -8.281695505674838
0.0  | 0.1 | 0.0 | 0   | -8.626229457431554  |  -8.626229457431554
0.0  | 0.1 | 0.0 | 0   | -9.32505355826198  |  -9.32505355826198
0.0  | 0.1 | 0.0 | 0   | -260.70613344320157  |  -260.70613344320157
0.0  | 0.1 | 0.0 | 0   | -290.8681222253365  |  -290.8681222253365
0.0  | 0.1 | 0.0 | 0   | -334.0243303812869  |  -334.0243303812869
0.0  | 0.1 | 0.0 | 0   | -3.5065779727708475  |  -3.5065779727708466
0.0  | 0.1 | 0.0 | 0   | -3.3553064586695625  |  -3.3553064586695616
0.0  | 0.1 | 0.0 | 0   | -3.0452586934897665  |  -3.045258693489767
0.0  | 0.2 | 0.0 | 0   | 6.819980193185756  |  6.819980193185758
0.0  | 0.2 | 0.0 | 0   | 7.1270889861

0.1  | 0.4 | 0.0 | 0   | -115.68561018501508  |  -115.68561018501507
0.1  | 0.4 | 0.0 | 0   | -95.68375592160194  |  -95.68375592160194
0.1  | 0.5 | 0.0 | 0   | -0.18402795705973482  |  -0.18402795705973482
0.1  | 0.5 | 0.0 | 0   | -0.1737046228731032  |  -0.17370462287310323
0.1  | 0.5 | 0.0 | 0   | -0.1636629903714963  |  -0.16366299037149631
0.1  | 0.5 | 0.0 | 0   | -0.03331741378892236  |  -0.03331741378892232
0.1  | 0.5 | 0.0 | 0   | -0.03236132308979705  |  -0.032361323089797056
0.1  | 0.5 | 0.0 | 0   | -0.03137692558192823  |  -0.03137692558192822
0.1  | 0.6 | 0.0 | 0   | -0.011131365814036339  |  -0.011131365814036346
0.1  | 0.6 | 0.0 | 0   | -0.01091692447411417  |  -0.01091692447411417
0.1  | 0.6 | 0.0 | 0   | -0.010691911345228289  |  -0.010691911345228296
0.1  | 0.6 | 0.0 | 0   | -0.004776994031481273  |  -0.004776994031481275
0.1  | 0.6 | 0.0 | 0   | -0.004708225006413041  |  -0.004708225006413038
0.1  | 0.6 | 0.0 | 0   | -0.00463538059330424  |  -0.00463538059330424
0.1  

0.2  | 0.1 | 0.0 | 0   | -0.807277504327221  |  -0.8072775043272199
0.2  | 0.1 | 0.0 | 0   | -0.7599729533123051  |  -0.7599729533123265
0.2  | 0.1 | 0.0 | 0   | -0.9497714892506601  |  -0.9497714892506649
0.2  | 0.1 | 0.0 | 0   | -1.016669389737745  |  -1.016669389737753
0.2  | 0.2 | 0.0 | 0   | -0.9157726098267429  |  -0.9157726098267269
0.2  | 0.2 | 0.0 | 0   | -1.0897589263637428  |  -1.0897589263637346
0.2  | 0.2 | 0.0 | 0   | -1.1588930019611028  |  -1.1588930019610981
0.2  | 0.2 | 0.0 | 0   | -1.7445654110057411  |  -1.7445654110057498
0.2  | 0.2 | 0.0 | 0   | -5.409842867859551  |  -5.40984286785955
0.2  | 0.2 | 0.0 | 0   | -8.934556032654168  |  -8.934556032654172
0.2  | 0.3 | 0.0 | 0   | -2.6148628275712804  |  -2.6148628275712884
0.2  | 0.3 | 0.0 | 0   | -3.3551350977405696  |  -3.355135097740572
0.2  | 0.3 | 0.0 | 0   | -5.012184090466177  |  -5.012184090466177
0.2  | 0.3 | 0.0 | 0   | -1.6459530548175045  |  -1.6459530548175043
0.2  | 0.3 | 0.0 | 0   | -1.2420669110147868 

0.3  | 0.6 | 0.0 | 0   | -0.002396045155065486  |  -0.0023960451550654872
0.3  | 0.6 | 0.0 | 0   | -0.0023104672893639955  |  -0.0023104672893639972
0.3  | 0.6 | 0.0 | 0   | -0.0014537831161771922  |  -0.0014537831161771928
0.3  | 0.6 | 0.0 | 0   | -0.0014100958389823208  |  -0.0014100958389823214
0.3  | 0.6 | 0.0 | 0   | -0.0013674646729587045  |  -0.0013674646729587051
0.3  | 0.7 | 0.0 | 0   | -0.0008893028978692587  |  -0.0008893028978692586
0.3  | 0.7 | 0.0 | 0   | -0.0008664544872531057  |  -0.0008664544872531061
0.3  | 0.7 | 0.0 | 0   | -0.0008439868597825675  |  -0.0008439868597825671
0.3  | 0.7 | 0.0 | 0   | -0.0005646606473466416  |  -0.0005646606473466419
0.3  | 0.7 | 0.0 | 0   | -0.0005521166616631237  |  -0.000552116661663123
0.3  | 0.7 | 0.0 | 0   | -0.0005397069821963584  |  -0.0005397069821963589
0.4  | 0.0 | 0.0 | 0   | -6.311477055114501  |  -6.311477055114501
0.4  | 0.0 | 0.0 | 0   | -45.089174678387735  |  -45.089174678387735
0.4  | 0.0 | 0.0 | 0   | -32.575999625994

0.5  | 0.2 | 0.0 | 0   | -0.013037109763174185  |  -0.013037109763174189
0.5  | 0.2 | 0.0 | 0   | -0.010415497011948409  |  -0.010415497011948412
0.5  | 0.2 | 0.0 | 0   | -0.009605179248641038  |  -0.009605179248641038
0.5  | 0.2 | 0.0 | 0   | -0.008885838963941194  |  -0.008885838963941194
0.5  | 0.3 | 0.0 | 0   | -0.006797365472439295  |  -0.006797365472439294
0.5  | 0.3 | 0.0 | 0   | -0.006335908280172441  |  -0.006335908280172442
0.5  | 0.3 | 0.0 | 0   | -0.005918427710108111  |  -0.0059184277101081125
0.5  | 0.3 | 0.0 | 0   | -0.004405539752904891  |  -0.004405539752904892
0.5  | 0.3 | 0.0 | 0   | -0.004144839254334823  |  -0.004144839254334821
0.5  | 0.3 | 0.0 | 0   | -0.0039050967207265425  |  -0.0039050967207265434
0.5  | 0.4 | 0.0 | 0   | -0.0028618221079489753  |  -0.002861822107948975
0.5  | 0.4 | 0.0 | 0   | -0.002713967931543858  |  -0.002713967931543856
0.5  | 0.4 | 0.0 | 0   | -0.0025761071158500832  |  -0.0025761071158500832
0.5  | 0.5 | 0.0 | 0   | -0.00187402679346181

0.6  | 0.6 | 0.0 | 0   | -0.00030395148175342844  |  -0.0003039514817534283
0.6  | 0.6 | 0.0 | 0   | -0.00029476863408661855  |  -0.00029476863408661865
0.6  | 0.6 | 0.0 | 0   | -0.00028590051703895696  |  -0.0002859005170389568
0.6  | 0.7 | 0.0 | 0   | -0.00022458722008249205  |  -0.0002245872200824921
0.6  | 0.7 | 0.0 | 0   | -0.0002184266958500794  |  -0.00021842669585007914
0.6  | 0.7 | 0.0 | 0   | -0.00021244839844090208  |  -0.000212448398440902
0.6  | 0.7 | 0.0 | 0   | -0.00016703918542919165  |  -0.00016703918542919152
0.6  | 0.7 | 0.0 | 0   | -0.00016287025657007705  |  -0.00016287025657007702
0.6  | 0.7 | 0.0 | 0   | -0.0001588069588300192  |  -0.00015880695883001934
0.7  | 0.0 | 0.0 | 0   | -0.0024733939956636153  |  -0.002473393995663612
0.7  | 0.0 | 0.0 | 0   | -0.002318483662613532  |  -0.002318483662613531
0.7  | 0.0 | 0.0 | 0   | -0.0021773214952672927  |  -0.002177321495267292
0.7  | 0.1 | 0.0 | 0   | -0.002411425166801292  |  -0.002411425166801291
0.7  | 0.1 | 0.0 | 0

In [6]:
errorsum = 0
print('Comparing modified version and original')
print('================================================================================================')
print(' kx  | ky  | qx  | qy  | rel error  ')
print('================================================================================================')
for i in range(0, 75, 5):
    for j in range(0, 75, 5):
        for k in range(1, 4, 1):
            print('%2.1f  | %2.1f | %2.1f | 0   |'%(i/100, j/100, k/100), relerror[i+j+k-1])
            errorsum = errorsum + relerror[i+j+k-1]
           
avgerror = errorsum / (900)            
print('================================================================================================')
print('The average error of modified function is ', avgerror)

Comparing modified version and original
 kx  | ky  | qx  | qy  | rel error  
0.0  | 0.0 | 0.0 | 0   | 0.0
0.0  | 0.0 | 0.0 | 0   | 0.0
0.0  | 0.0 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 1.5543122344752192e-15
0.0  | 0.1 | 0.0 | 0   | 1.887379141862766e-15
0.0  | 0.1 | 0.0 | 0   | 2.9976021664879227e-15
0.0  | 0.2 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.2 | 0.0 | 0   | 0.0
0.0  | 0.2 | 0.0 | 0   | 0.0
0.0  | 0.2 | 0.0 | 0   | 7.549516567451064e-15
0.0  | 0.2 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.2 | 0.0 | 0   | 1.099120794378905e-14
0.0  | 0.3 | 0.0 | 0   | 5.329070518200751e-15
0.0  | 0.3 | 0.0 | 0   | 0.0
0.0  | 0.3 | 0.0 | 0   | 1.4432899320127035e-15
0.0  | 0.3 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.3 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.3 | 0.0 | 0   | 1.1102230246

# Ensure that treating the arrays can be implemented with the loops above

In [7]:
N = 2

sing = np.arange(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
dbl = np.arange(-(N - 1), (N - 1) + 1, 1)

# rng1 = range(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
print('sing:')
print(sing)
print('\nsing as loop:')
k = -(N - 1) / 2    
while(k < ((N - 1) / 2 + 1)):
    print(k)
    k = k + 1
        
print('\ndbl:')
print(dbl)
print('\ndbl as range:')
for i in range(-(N - 1), N, 1):
    print(i)

sing:
[-0.5  0.5]

sing as loop:
-0.5
0.5

dbl:
[-1  0  1]

dbl as range:
-1
0
1


# Checking the consistency of math and numpy log function

In [8]:
A = 4  # hbar^2/(2m)=4 evAA^2 (for free electron mass)
eE0 = rati * ((hOmg) ** 2) / (2 * np.sqrt(A * mu))

KT = 1 * 10 ** (-6)
shift = A * (eE0 / hOmg) ** 2

Gamm = 0.005  # Gamma in eV.
hOmg = 0.5  # Photon energy eV

ek = A * (math.sqrt((0.1) ** 2 + (0.1) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2

z = Gamm ** 2 + (ek - hOmg / 2 + hOmg * -0.5) ** 2

mlog = math.log(z)
nplog = np.log(z)

print('math.log result: ', mlog)
print('np.log result: ', nplog)

arrtest0 = [complex(0,mlog)]
arrtest1 = [complex(0,nplog)]

print('math.log result in array: ', arrtest0)
print('np.log result in array: ', arrtest1)

mlog = math.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * -0.5) ** 2)
nplog = np.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * -0.5) ** 2)

print('math.log result: ', mlog)
print('np.log result: ', nplog)

math.log result:  -2.0223879944493093
np.log result:  -2.0223879944493093
math.log result in array:  [-2.0223879944493093j]
np.log result in array:  [-2.0223879944493093j]
math.log result:  -2.0223879944493093
np.log result:  -2.0223879944493093


# Testing my Bessel function for various values of v and z

Ensuring that the number of terms makes the relative error equal to machine epsilon.

In [9]:
def my_Besselv(v, z):
    # WILL NOT WORK IF v IS NOT AN INTEGER
    # Conditional to handle case of negative v.
    if(v < 0):
        v = abs(v)
        resultsign = (-1) ** v
    else:
        resultsign = 1
    result = 0    
    # Loop to construct Bessel series sum.
    for n in range(0,20):
        sign = (-1)**n
        exp = 2 * n + v
        term = z ** exp
        r = n + v + 1
        if(r == 0):
            r = 1e-15
        denom = math.gamma(r)
        denom = denom * math.factorial(n)
        denom = denom * (2 ** exp)
        term = term / denom * sign
        # print('for ', n, ': ',term)
        result = result + term
        
    return result * resultsign

relerror = np.zeros(78)
l = 0

print('Comparing scipy library Bessel function and my Bessel function:')
print('================================================================================================')
print(' v | z | scipy result |   my result   | rel error')
print('================================================================================================')
for i in range(-3,3):
    for j in range(20,33):
        myres = my_Besselv(i, j/10)
        scires = scipy.special.jv(i, j/10)
        relerror[l] = abs((scires / myres) - 1)
        print('%3d|%3.1f|%14E|%15E|%E' % (i, j/10, scires, myres, relerror[l]))
        l = l + 1
print('================================================================================================')

avgerr = math.fsum(relerror) / l
print('Average relative error: ', avgerr)

Comparing scipy library Bessel function and my Bessel function:
 v | z | scipy result |   my result   | rel error
 -3|2.0| -1.289432E-01|  -1.289432E-01|2.220446E-16
 -3|2.1| -1.452767E-01|  -1.452767E-01|0.000000E+00
 -3|2.2| -1.623255E-01|  -1.623255E-01|2.220446E-16
 -3|2.3| -1.799789E-01|  -1.799789E-01|4.440892E-16
 -3|2.4| -1.981148E-01|  -1.981148E-01|2.220446E-16
 -3|2.5| -2.166004E-01|  -2.166004E-01|4.440892E-16
 -3|2.6| -2.352938E-01|  -2.352938E-01|2.220446E-16
 -3|2.7| -2.540453E-01|  -2.540453E-01|4.440892E-16
 -3|2.8| -2.726986E-01|  -2.726986E-01|2.220446E-16
 -3|2.9| -2.910926E-01|  -2.910926E-01|2.220446E-16
 -3|3.0| -3.090627E-01|  -3.090627E-01|2.220446E-16
 -3|3.1| -3.264428E-01|  -3.264428E-01|2.220446E-16
 -3|3.2| -3.430664E-01|  -3.430664E-01|3.330669E-16
 -2|2.0|  3.528340E-01|   3.528340E-01|0.000000E+00
 -2|2.1|  3.746236E-01|   3.746236E-01|1.110223E-16
 -2|2.2|  3.950587E-01|   3.950587E-01|2.220446E-16
 -2|2.3|  4.139146E-01|   4.139146E-01|2.220446E-16
 -